In [1]:
from adclick import data_loader
df = data_loader.load_data("/Users/solene.leboucher/Desktop/DSBA/MLOps/ClickTraining.csv")
df.head()

/Users/solene.leboucher/dsba-platform/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Unnamed: 0,Region,Daytime,Carrier,Time_On_Previous_Website,Weekday,Social_Network,Number_of_Previous_Orders,Clicks_Conversion,Restaurant_Type
0,18847,Alsace and East France,0.223254,SFR,8.066935,Saturday,Instagram,3,1,French
1,18895,West France,0.734393,Bouygues,257.567079,Saturday,Twitter,3,1,French
2,2986,Paris,0.781317,SFR,1427.640826,Wednesday,Instagram,6,1,French
3,1842,Paris,0.310846,SFR,1606.262115,Wednesday,Instagram,4,1,Burger
4,3371,North France,0.910376,SFR,480.234887,Sunday,Facebook,2,1,Groceries


In [2]:
from adclick.preprocessing import DataPreprocessor
preprocessor = DataPreprocessor()
df_preprocessed = preprocessor.preprocess(df)
df_preprocessed.head()

,Unnamed: 0,Daytime,Time_On_Previous_Website,Number_of_Previous_Orders,Clicks_Conversion,Region_North France,Region_Paris,Region_Rest of France,Region_South France,Region_West France,...,Social_Network_Instagram,Social_Network_Twitter,Restaurant_Type_French,Restaurant_Type_Groceries,Restaurant_Type_Kebab,Restaurant_Type_Sushi,Restaurant_Type_Unknown,DaytimeXOrders,InvTimeonPrevSite,Is_Weekend
0,18847,0.223254,8.066935,3,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0.669762,0.110291,1
1,18895,0.734393,257.567079,3,1,0,0,0,0,1,...,0,1,1,0,0,0,0,2.203180,0.003867,1
2,2986,0.781317,1427.640826,6,1,0,1,0,0,0,...,1,0,1,0,0,0,0,4.687902,0.000700,0
3,1842,0.310846,1606.262115,4,1,0,1,0,0,0,...,1,0,0,0,0,0,0,1.243385,0.000622,0
4,3371,0.910376,480.234887,2,1,1,0,0,0,0,...,0,0,0,1,0,0,0,1.820752,0.002078,1


In [3]:
df_preprocessed.drop(columns = "Unnamed: 0", inplace = True)
df_preprocessed.head()

,Daytime,Time_On_Previous_Website,Number_of_Previous_Orders,Clicks_Conversion,Region_North France,Region_Paris,Region_Rest of France,Region_South France,Region_West France,Carrier_Free,...,Social_Network_Instagram,Social_Network_Twitter,Restaurant_Type_French,Restaurant_Type_Groceries,Restaurant_Type_Kebab,Restaurant_Type_Sushi,Restaurant_Type_Unknown,DaytimeXOrders,InvTimeonPrevSite,Is_Weekend
0,0.223254,8.066935,3,1,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0.669762,0.110291,1
1,0.734393,257.567079,3,1,0,0,0,0,1,0,...,0,1,1,0,0,0,0,2.203180,0.003867,1
2,0.781317,1427.640826,6,1,0,1,0,0,0,0,...,1,0,1,0,0,0,0,4.687902,0.000700,0
3,0.310846,1606.262115,4,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1.243385,0.000622,0
4,0.910376,480.234887,2,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1.820752,0.002078,1


In [4]:
from adclick.model_training import ModelTrainer
trainer = ModelTrainer()

numerical_cols = ['Time_On_Previous_Website',	'Number_of_Previous_Orders', 'Daytime', 'DaytimeXOrders', 'InvTimeonPrevSite'] 
df_preprocessed = trainer.apply_smote(df_preprocessed, "Clicks_Conversion", numerical_cols)
df_preprocessed.columns = df_preprocessed.columns.str.replace(" ", "_", regex=True)

In [10]:
# Split the data between train and test sets
X_train, X_test, y_train, y_test = trainer.train_test_split(df_preprocessed, "Clicks_Conversion")

# Logistic Regression
lr = trainer.train_logistic_regression(X_train, y_train)

In [11]:
from adclick import model_evaluation
model_evaluation.evaluate_model(lr, X_test, y_test)

Accuracy: 0.9011
Precision: 0.9258
Recall: 0.9474
F1 Score: 0.9365
ROC-AUC Score: 0.9598


{'Accuracy': 0.9010877814318239,
 'Precision': 0.925771208226221,
 'Recall': 0.9473857283788227,
 'F1 Score': 0.9364537623923289,
 'ROC-AUC Score': np.float64(0.9597835430404356)}

In [13]:
# Random Forest 
rf = trainer.tune_random_forest(X_train, y_train)

model_evaluation.evaluate_model(rf, X_test, y_test)

Accuracy: 0.9201
Precision: 0.9377
Recall: 0.9599
F1 Score: 0.9487
ROC-AUC Score: 0.9708


{'Accuracy': 0.9200607133822414,
 'Precision': 0.9376806938644394,
 'Recall': 0.9598816178888524,
 'F1 Score': 0.9486512837179071,
 'ROC-AUC Score': np.float64(0.9707791037112676)}

In [14]:
lgbm = trainer.tune_lgbm(X_train, y_train)

[I 2025-03-30 15:56:12,183] A new study created in memory with name: no-name-860983d8-9d50-49cb-9d40-4b15ab88e6a9


[LightGBM] [Warning] lambda_l2 is set=3.477460065272316, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.477460065272316
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l2 is set=3.477460065272316, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.477460065272316
[LightGBM] [Info] Number of positive: 9728, number of negative: 2919
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000358 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1076
[LightGBM] [Info] Number of data points in the train set: 12647, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.769194 -> initscore=1.203767
[LightGBM] [Info] Start training from score 1.203767
[LightGBM] [Warning] lambda_l2 is set=3.477460065272316, reg_lambda=0.0 will be ignored. Current value

In [12]:
model_evaluation.evaluate_model(lgbm, X_test, y_test)

[LightGBM] [Warning] lambda_l2 is set=0.18920245161815494, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.18920245161815494
[LightGBM] [Warning] lambda_l2 is set=0.18920245161815494, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.18920245161815494
Accuracy: 0.9309
Precision: 0.9511
Recall: 0.9596
F1 Score: 0.9553
ROC-AUC Score: 0.9785


{'Accuracy': 0.9309385277004807,
 'Precision': 0.9511082138200783,
 'Recall': 0.95955277869122,
 'F1 Score': 0.9553118349975446,
 'ROC-AUC Score': np.float64(0.9784909598066195)}